GROUP 6\
1.\
a. Objective of project: Predict BTC/USDT price trend in the next 1 hour\
b. Method of analysis: LSTMs, regression\
c. Dataset:\
-Independent variables: Open, Close, High, Low, Volume BTC, Volume USDT\
-Dependent variable: Close (price)\
-Column descriptions:\
● Unix: Unix timestamp is a digital standard for denoting an exact, universal time\
● Date and Time: Date and time of price with time interval of 1 hour\
● Open: BTC/USDT price first trade upon the start of the time period\
● Close: BTC/USDT price at the end of the time period\
● High: Highest BTC/USDT price during the 1 hour time period\
● Low: Lowest BTC/USDT price during the 1 hour time period\
● Volume BTC: Measure of how much of a given BTC has been traded in the time
period\
● Volume USDT: Measure of how much of a given USDT has been traded in
the time period\
● Tradecount: Number of trades

In [1]:
import pyspark.sql.functions as fn

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1626333902105_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2. Data Cleaning

2a. Reading from CSV

In [2]:
df = (
spark
.read
.csv(
's3://group6project1/BTCUSDT Binance.csv'
, header=True
, inferSchema=True)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o86.csv.
: java.io.IOException: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: 7MNH46ZXYSG5NSDT; S3 Extended Request ID: qjEaH5ClDGbGVsuZJblJ/qr0u2oIRvtCYl2XK0xZU8R4TlUakv++gVSUtbs56h5OoTrgGwVQExg=), S3 Extended Request ID: qjEaH5ClDGbGVsuZJblJ/qr0u2oIRvtCYl2XK0xZU8R4TlUakv++gVSUtbs56h5OoTrgGwVQExg=
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.list(Jets3tNativeFileSystemStore.java:329)
	at com.amazon.ws.emr.hadoop.fs.s3n.S3NativeFileSystem.getFileStatus(S3NativeFileSystem.java:613)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1440)
	at com.amazon.ws.emr.hadoop.fs.EmrFileSystem.exists(EmrFileSystem.java:357)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSo

In [3]:
#Drop symbol column because all data is about BTC/USDT prices:
df = df.drop('symbol')
df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



In [4]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



2b. Duplicated rows?

In [5]:
#Count total number of rows and total number of unique rows to check for duplicates / exact duplicates:
df.count(), df.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



This indicates that all rows are distinct.

In [6]:
#Count total number of unix and total number of unique unix to check for duplicates:
df.agg(
      fn.count('unix').alias('CountOfUnix')
    , fn.countDistinct('unix').alias('CountOfDistinctUnix')
).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



This indicates that all unix are distinct.

2c. Missing observations?

In [7]:
#Count percentage of missing values in each column:
for k, v in sorted(
    df.agg(*[
               (1 - (fn.count(c) / fn.count('*')))
                    .alias(c + '_miss')
               for c in df.columns
           ])
        .collect()[0]
        .asDict()
        .items()
    , key=lambda el: el[1]
    , reverse=True
):
    print(k, v)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



Because tradecount column has 78.53% of missing values, it will be dropped. 

In [8]:
#Drop tradecount column:
no_tradecount = df.select([col for col in df.columns if col != 'tradecount'])
no_tradecount.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df' is not defined
Traceback (most recent call last):
NameError: name 'df' is not defined



In [9]:
#Count the number of rows with missing values
no_tradecount.rdd.map(
           lambda row: (
                 row['unix']
               , sum([c == None for c in row])
           )
        )\
        .filter(lambda el: el[1] >= 1)\
        .count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'no_tradecount' is not defined
Traceback (most recent call last):
NameError: name 'no_tradecount' is not defined



There is 1 row with missing values.

In [10]:
#Count the number of missing values in that row:
(
    spark.createDataFrame(
        no_tradecount.rdd.map(
           lambda row: (
                 row['unix']
               , sum([c == None for c in row])
           )
        )
        .filter(lambda el: el[1] >= 1)
        .collect()
        ,['unix', 'CountMissing']
    )
    .orderBy('CountMissing', ascending=False)
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'no_tradecount' is not defined
Traceback (most recent call last):
NameError: name 'no_tradecount' is not defined



There is 1 missing value in that row.

In [11]:
#View the row with missing values:
(
    no_tradecount
    .where('unix == 12628117525348800')
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'no_tradecount' is not defined
Traceback (most recent call last):
NameError: name 'no_tradecount' is not defined



In [12]:
#Drop rows that have less than thresh NON-NULL values:
df_clean = no_tradecount.dropna(thresh = 9)
no_tradecount.count(), df_clean.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'no_tradecount' is not defined
Traceback (most recent call last):
NameError: name 'no_tradecount' is not defined



One row is dropped.

In [13]:
#Making sure the row with missing value is gone
(
    df_clean
    .where('unix == 12628117525348800')
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_clean' is not defined
Traceback (most recent call last):
NameError: name 'df_clean' is not defined



The row with missing value is gone now.

3. Descriptive statistics

In [14]:
#Describe numeric features:
numeric = ['open','high','low','close','Volume BTC','Volume USDT']
descriptive_stats = df_clean.describe(numeric)
descriptive_stats.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_clean' is not defined
Traceback (most recent call last):
NameError: name 'df_clean' is not defined



- Count: Each column has 36,398 values.\
- Mean: The means of open, high, low, close prices range from 13,028 to approximately 13,204 dollars. The difference between the means of high and low in an 1 hour is $176. On average, 2,053 BTCs and 34,907,045 USDT are being traded during the time period.\
- Standard deviation: Standard deviation gives a view of how widely values are dispersed from the average value. For example, if standard deviation of price is high, this indicates the price volatility. On the other hand, if the closing prices are close and do not fluctuate much from the average mean price, standard deviation is less and the markets volatility is considered less as well.\
- Min: This gives the min/lowest value of each column.\
- Max: This gives the max/highest value of each column.

Because unix and date data do not show anything meaningful, we do not do descriptive statistics on them.

4. Visualization

Histogram of Close

In [15]:
#generate bins and count of each bin
histogram_close = (
    df_clean
    .select('close')
    .rdd
    .flatMap(lambda record: record)
    .histogram(20)
)
histogram_close
#display bin ranges
for i in range(len(histogram_close[0])-1):
    print('[' + str(round(histogram_close[0][i],2))
        + ',' + str(round(histogram_close[0][i+1],2))
          + ')'
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_clean' is not defined
Traceback (most recent call last):
NameError: name 'df_clean' is not defined



In [16]:
%%spark -o hist_close 
hist_close =  spark.createDataFrame(
    list(zip(*histogram_close)), 
    ['close', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_close.toPandas().set_index('close'
            ).plot(kind='bar')
%matplot plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'histogram_close' is not defined
Traceback (most recent call last):
NameError: name 'histogram_close' is not defined



According to this histogram, close prices mostly range from 2,919 to 12,167. Some are between 30,665 and 39,913, and between 46,079 and 61,494. Close prices range from 6,001 to 9,084 for approximately 12,000 times. The distribution of close prices is right-skewed. This indicates that the likelihood of an extremely large price is somewhat greater than the likelihood of an extremely low price. The close prices of BTC/USDT are not normally distributed.

Histogram of High

In [17]:
histogram_high = (
    df_clean
    .select('high')
    .rdd
    .flatMap(lambda record: record)
    .histogram(20)
)
for i in range(len(histogram_high[0])-1):
    print('[' + str(round(histogram_high[0][i],2))
        + ',' + str(round(histogram_high[0][i+1],2))
          + ')'
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_clean' is not defined
Traceback (most recent call last):
NameError: name 'df_clean' is not defined



In [18]:
%%spark -o hist_high
hist_high =  spark.createDataFrame(
    list(zip(*histogram_high)), 
    ['high', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_high.toPandas().set_index('high'
            ).plot(kind='bar')
%matplot plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'histogram_high' is not defined
Traceback (most recent call last):
NameError: name 'histogram_high' is not defined



High prices mostly range from 2,950 to 12,235. Some are between 30,806 and 40,092, and between 46,282 and 61,758. High prices range from 6,045 to 9,140 for approximately 12,000 times. The distribution of high prices is right-skewed. This indicates that the likelihood of an extremely large price is somewhat greater than the likelihood of an extremely low price. The high prices of BTC/USDT are not normally distributed.

Histogram of low

In [19]:
histogram_low = (
    df_clean
    .select('low')
    .rdd
    .flatMap(lambda record: record)
    .histogram(20)
)
for i in range(len(histogram_low[0])-1):
    print('[' + str(round(histogram_low[0][i],2))
        + ',' + str(round(histogram_low[0][i+1],2))
          + ')'
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_clean' is not defined
Traceback (most recent call last):
NameError: name 'df_clean' is not defined



In [20]:
%%spark -o hist_low
hist_low =  spark.createDataFrame(
    list(zip(*histogram_low)), 
    ['low', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_low.toPandas().set_index('low'
            ).plot(kind='bar')
%matplot plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'histogram_low' is not defined
Traceback (most recent call last):
NameError: name 'histogram_low' is not defined



Low prices mostly range from 2,817 to 12,036. Some are between 30,475 and 39,694, and between 45,841 and 61,206. Low prices range from 5,890 to 8,963 for approximately 12,000 times. The distribution of low prices is right-skewed. This indicates that the likelihood of an extremely large price is somewhat greater than the likelihood of an extremely low price. The low prices of BTC/USDT are not normally distributed.

Histogram of open

In [21]:
histogram_open = (
    df_clean
    .select('open')
    .rdd
    .flatMap(lambda record: record)
    .histogram(20)
)
for i in range(len(histogram_open[0])-1):
    print('[' + str(round(histogram_open[0][i],2))
        + ',' + str(round(histogram_open[0][i+1],2))
          + ')'
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_clean' is not defined
Traceback (most recent call last):
NameError: name 'df_clean' is not defined



In [22]:
%%spark -o hist_open
hist_open =  spark.createDataFrame(
    list(zip(*histogram_open)), 
    ['open', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_open.toPandas().set_index('open'
            ).plot(kind='bar')
%matplot plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'histogram_open' is not defined
Traceback (most recent call last):
NameError: name 'histogram_open' is not defined



Open prices mostly range from 2,919 to 12,167. Some are between 30,665 and 39,913, and between 46,079 and 61,494. Open prices range from 6,001 to 9,084 for approximately 12,000 times. The distribution of open prices is right-skewed. This indicates that the likelihood of an extremely large price is somewhat greater than the likelihood of an extremely low price. The low prices of BTC/USDT are not normally distributed.

Histogram of Volume BTC

In [23]:
histogram_volBTC = (
    df_clean
    .select('Volume BTC')
    .rdd
    .flatMap(lambda record: record)
    .histogram(30)
)
for i in range(len(histogram_volBTC[0])-1):
    print('[' + str(round(histogram_volBTC[0][i],2))
        + ',' + str(round(histogram_volBTC[0][i+1],2))
          + ')'
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_clean' is not defined
Traceback (most recent call last):
NameError: name 'df_clean' is not defined



In [24]:
%%spark -o hist_volBTC
hist_volBTC =  spark.createDataFrame(
    list(zip(*histogram_volBTC)), 
    ['Volume BTC', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_volBTC.toPandas().set_index('Volume BTC'
            ).plot(kind='bar')
%matplot plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'histogram_volBTC' is not defined
Traceback (most recent call last):
NameError: name 'histogram_volBTC' is not defined



The distribution of Volume BTC is positively skewed. Most Volume BTC are between 0 and 3,150. In 1 hour, approximately 0 to 1,575 BTC were traded with the higest frequency of 19,000.

Histogram of USDT

In [25]:
histogram_volUSDT = (
    df_clean
    .select('Volume USDT')
    .rdd
    .flatMap(lambda record: record)
    .histogram(30)
)
for i in range(len(histogram_volUSDT[0])-1):
    print('[' + str(round(histogram_volUSDT[0][i],2))
        + ',' + str(round(histogram_volUSDT[0][i+1],2))
          + ')'
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_clean' is not defined
Traceback (most recent call last):
NameError: name 'df_clean' is not defined



In [26]:
%%spark -o hist_volUSDT
hist_volUSDT =  spark.createDataFrame(
    list(zip(*histogram_volUSDT)), 
    ['Volume USDT', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_volUSDT.toPandas().set_index('Volume USDT'
            ).plot(kind='bar')
%matplot plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'histogram_volUSDT' is not defined
Traceback (most recent call last):
NameError: name 'histogram_volUSDT' is not defined



The distribution of Volume USDT is positively skewed. Most Volume USDT are between 0 and 50,482,160 with the higest frequency of 30,000.

In [27]:
df_pd = df_clean.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_clean' is not defined
Traceback (most recent call last):
NameError: name 'df_clean' is not defined



In [28]:
#Scatter Plot for Low and Volume BTC
import matplotlib.pyplot as plt
plt.cla()
plt.scatter(df_pd['low']
         , df_pd['Volume BTC'], 
            label="Relationship between low and Volume BTC")
plt.xlabel("low")
plt.ylabel("Volume BTC")
plt.legend(loc='upper left')
%matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_pd' is not defined
Traceback (most recent call last):
NameError: name 'df_pd' is not defined



In [29]:
#Scatter Plot for High and Volume BTC
import matplotlib.pyplot as plt
plt.cla()
plt.scatter(df_pd['high']
         , df_pd['Volume BTC'], 
            label="Relationship between high and Volume BTC")
plt.xlabel("high")
plt.ylabel("Volume BTC")
plt.legend(loc='upper left')
%matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_pd' is not defined
Traceback (most recent call last):
NameError: name 'df_pd' is not defined



The relationship between High and Volume BTC is cointigrated. Most of the time, a big increase in High will cause Volume BTC to increase and a big decrease in High will cause Volume BTC to decrease. 

In [30]:
#Scatter Plot for close and Volume BTC
import matplotlib.pyplot as plt
plt.cla()
plt.scatter(df_pd['close']
         , df_pd['Volume BTC'], 
            label="Relationship between close and Volume BTC")
plt.xlabel("close")
plt.ylabel("Volume BTC")
plt.legend(loc='upper left')
%matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_pd' is not defined
Traceback (most recent call last):
NameError: name 'df_pd' is not defined



The relationship between Close and Volume BTC is cointigrated. Most of the time, a big increase in Close will cause Volume BTC to increase and a big decrease in Close will cause Volume BTC to decrease.

In [31]:
#Scatter Plot for Volume BTC and Volume USDT
import matplotlib.pyplot as plt
plt.cla()
plt.scatter(df_pd['Volume BTC']
         , df_pd['Volume USDT'], 
            label="Relationship between Volume BTC and Volume USDT")
plt.xlabel("Volume BTC")
plt.ylabel("Volume USDT")
plt.legend(loc='upper left')
%matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_pd' is not defined
Traceback (most recent call last):
NameError: name 'df_pd' is not defined



5. Outlier analysis

Identify outliers based on visualization:\
As we can see from the charts, the outliers of Volume BTC are those above 30000, the outliers of Volume USDT are those above 1000000000, the outliers of prices (open, close, high, low) are those above 59000.

In [32]:
#Filter outliers
df_no_outlier = df_clean.where((df_clean['Volume BTC'] < 30000) & (df_clean['Volume USDT'] < 1000000000) & (df_clean['open'] < 59000) & (df_clean['close'] < 59000) &(df_clean['high'] < 59000) &(df_clean['low'] < 59000))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_clean' is not defined
Traceback (most recent call last):
NameError: name 'df_clean' is not defined



Approximately 1% of the data are outliers and were eliminated.

6.

In [33]:
#Rerun descriptive statistics
stat_no_outlier = df_no_outlier.describe(numeric)
stat_no_outlier.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_no_outlier' is not defined
Traceback (most recent call last):
NameError: name 'df_no_outlier' is not defined



Redraw histograms: Done in Jupyter Notebook named 'Group 6 - Visualization'.

Histogram of numeric features

Histogram of close price

In [34]:
#generate bins and count of each bin
histogram_close = (
    stat_no_outlier
    .select('close')
    .rdd
    .flatMap(lambda record: record)
    .histogram(20)
)
histogram_close
#display bin ranges
for i in range(len(histogram_close[0])-1):
    print('[' + str(round(histogram_close[0][i],2))
        + ',' + str(round(histogram_close[0][i+1],2))
          + ')'
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'stat_no_outlier' is not defined
Traceback (most recent call last):
NameError: name 'stat_no_outlier' is not defined



In [35]:
%%spark -o hist_close 
hist_close =  spark.createDataFrame(
    list(zip(*histogram_close)), 
    ['close', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_close.toPandas().set_index('close'
            ).plot(kind='bar')
%matplot plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'histogram_close' is not defined
Traceback (most recent call last):
NameError: name 'histogram_close' is not defined



Histogram of open price

In [ ]:
#generate bins and count of each bin
histogram_open = (
    stat_no_outlier
    .select('open')
    .rdd
    .flatMap(lambda record: record)
    .histogram(20)
)
histogram_open
#display bin ranges
for i in range(len(histogram_open[0])-1):
    print('[' + str(round(histogram_open[0][i],2))
        + ',' + str(round(histogram_open[0][i+1],2))
          + ')'
         )

In [ ]:
%%spark -o hist_open 
hist_open =  spark.createDataFrame(
    list(zip(*histogram_open)), 
    ['open', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_close.toPandas().set_index('open'
            ).plot(kind='bar')
%matplot plt

Histogram of low price

In [ ]:
#generate bins and count of each bin
histogram_low = (
    stat_no_outlier
    .select('low')
    .rdd
    .flatMap(lambda record: record)
    .histogram(20)
)
histogram_low
#display bin ranges
for i in range(len(histogram_low[0])-1):
    print('[' + str(round(histogram_low[0][i],2))
        + ',' + str(round(histogram_low[0][i+1],2))
          + ')'
         )

In [ ]:
%%spark -o hist_low 
hist_low =  spark.createDataFrame(
    list(zip(*histogram_low)), 
    ['low', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_close.toPandas().set_index('low'
            ).plot(kind='bar')
%matplot plt

Histogram of high price

In [ ]:
#generate bins and count of each bin
histogram_high = (
    stat_no_outlier
    .select('high')
    .rdd
    .flatMap(lambda record: record)
    .histogram(20)
)
histogram_high
#display bin ranges
for i in range(len(histogram_high[0])-1):
    print('[' + str(round(histogram_high[0][i],2))
        + ',' + str(round(histogram_high[0][i+1],2))
          + ')'
         )

In [ ]:
%%spark -o hist_high 
hist_high =  spark.createDataFrame(
    list(zip(*histogram_high)), 
    ['high', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_high.toPandas().set_index('high'
            ).plot(kind='bar')
%matplot plt

Histogram of volume BTC

In [ ]:
#generate bins and count of each bin
histogram_volumeBTC = (
    stat_no_outlier
    .select('Volume BTC')
    .rdd
    .flatMap(lambda record: record)
    .histogram(20)
)
histogram_volumeBTC
#display bin ranges
for i in range(len(histogram_volumeBTC[0])-1):
    print('[' + str(round(histogram_volumeBTC[0][i],2))
        + ',' + str(round(histogram_volumeBTC[0][i+1],2))
          + ')'
         )

In [ ]:
%%spark -o hist_volumeBTC 
hist_volumeBTC =  spark.createDataFrame(
    list(zip(*histogram_volumeBTC)), 
    ['Volume BTC', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_high.toPandas().set_index('Volume BTC'
            ).plot(kind='bar')
%matplot plt

Histogram of Volume USDT

In [ ]:
#generate bins and count of each bin
histogram_volumeUSDT = (
    stat_no_outlier
    .select('Volume USDT')
    .rdd
    .flatMap(lambda record: record)
    .histogram(20)
)
histogram_volumeUSDT
#display bin ranges
for i in range(len(histogram_volumeUSDT[0])-1):
    print('[' + str(round(histogram_volumeUSDT[0][i],2))
        + ',' + str(round(histogram_volumeUSDT[0][i+1],2))
          + ')'
         )

In [ ]:
%%spark -o hist_volumeUSDT
hist_volumeUSDT =  spark.createDataFrame(
    list(zip(*histogram_volumeUSDT)), 
    ['Volume USDT', 'counts'])
import matplotlib
import matplotlib.pyplot as plt
hist_high.toPandas().set_index('Volume USDT'
            ).plot(kind='bar')
%matplot plt